Pre-processing Proverbs and determining now to match them to prayers.

In [146]:
with open("biblical_data/proverbs.txt") as f:
    proverbs = f.readlines()

In [147]:
proverbs = [p.rstrip("\n").strip() for p in proverbs]

In [148]:
proverbs_short = [p for p in proverbs if len(p)<=140]

In [149]:
len(proverbs_short)

851

In [150]:
import itertools
import re
proverbs_long = [p for p in proverbs if len(p)>140]
for p in proverbs_long:
    frangibles = re.split(r"[?!;:]",p)
    frangibles = [f.strip() for f in frangibles]
    for f in frangibles:
        if len(f)>30:
            if f[0].isupper():
                proverbs_short.append(f)
    

In [151]:
len(proverbs_short)

1102

In [152]:
proverbs = proverbs_short

In [153]:
print("****Loading sbert model")
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
print("****Loaded")
import torch


****Loading sbert model
****Loaded


In [154]:
## embed
proverbs_embedded = model.encode(proverbs_short, convert_to_tensor=True) ## set up for sbert

In [155]:
proverbs_embedded[0]

tensor([-1.0356e-01,  8.4312e-01,  4.1678e-01,  2.9132e-01,  2.1492e-01,
        -2.3340e-01,  1.3825e-01, -1.0411e+00,  4.0200e-01, -3.9351e-01,
         2.4300e-01,  9.3490e-01,  6.7175e-02,  7.9243e-01, -2.6569e-01,
         4.7522e-02,  1.9867e+00, -1.1379e-01, -6.9688e-01, -5.5760e-01,
        -5.2349e-01,  2.9936e-01,  2.1467e-01,  1.0238e+00,  7.8902e-02,
         8.1300e-01, -7.0217e-02,  7.4062e-03,  3.2122e-01,  2.6714e-01,
         6.4935e-01,  4.3660e-01,  7.1065e-01, -1.8607e-01,  3.1868e-01,
        -7.5774e-03, -6.0237e-03,  3.5018e-01,  5.4521e-01, -6.4244e-02,
        -4.6972e-01, -3.1875e-02,  1.2070e-01,  4.4207e-01,  5.0091e-01,
        -5.0167e-01, -1.3289e-01,  5.0545e-01, -8.2512e-01, -1.2926e-01,
        -6.4990e-01,  8.5707e-01, -1.3086e-02,  2.7571e-01,  2.1861e-01,
        -1.0715e-01, -2.0489e-01,  2.2103e-01,  2.7359e-01, -1.0180e-01,
         2.7444e-01,  3.9330e-01, -5.8393e-01, -4.3269e-01, -7.2968e-01,
        -3.8730e-03, -4.1149e-01,  4.8001e-01,  5.3

In [156]:
torch.save(proverbs_embedded,"proverbs_embedded.pt")
proverbs_embedded = torch.load("proverbs_embedded.pt")

In [157]:
import json
with open("proverbs_paired_to_tensors.json","w") as f:
    json.dump(proverbs,f)
    
with open("proverbs_paired_to_tensors.json","r") as f:
    proverbs = json.load(f)

In [158]:
import random

def get_close_proverb_sbert(prayer):
    """
    calculates the most similar need from my custom needs using sent2vec
    returns dict with need and similarity score
    """
    query_embedding = model.encode(prayer, convert_to_tensor=True)
    #return query_embedding
    cos_scores = util.pytorch_cos_sim(query_embedding, proverbs_embedded)[0]
    cos_scores = cos_scores.cpu()
    score,top_match_indx = torch.topk(cos_scores, k=20)
    matching_proverbs = [proverbs[i] for i in top_match_indx]
    print(matching_proverbs)
    while True:
        if random.choice([False,False,False,False,True]):
            return matching_proverbs[0]
        else:
            last = matching_proverbs.pop(0)
            matching_proverbs.append(last)

In [159]:
get_close_proverb_sbert("May you sleep in the garden of love.")

['When thou liest down, thou shalt not be afraid: yea, thou shalt lie down, and thy sleep shall be sweet.', 'Yet a little sleep, a little slumber, a little folding of the hands to sleep', 'Yet a little sleep, a little slumber, a little folding of the hands to sleep', 'Come, let us take our fill of love until the morning: let us solace ourselves with loves.', 'Correct thy son, and he shall give thee rest; yea, he shall give delight unto thy soul.', 'So she caught him, and kissed him, and with an impudent face said unto him, I have peace offerings with me; this day have I payed my vows.', 'My son, eat thou honey, because it is good', 'He that gathereth in summer is a wise son: but he that sleepeth in harvest is a son that causeth shame.', 'Forsake her not, and she shall preserve thee: love her, and she shall keep thee.', 'Go thy way, eat thy bread with joy, and drink thy wine with a merry heart; for God now accepteth thy works.', 'He that loveth pureness of heart, for the grace of his li

'So she caught him, and kissed him, and with an impudent face said unto him, I have peace offerings with me; this day have I payed my vows.'